<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goroutings-and-Channels" data-toc-modified-id="Goroutings-and-Channels-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Goroutings and Channels</a></span><ul class="toc-item"><li><span><a href="#Goroutines" data-toc-modified-id="Goroutines-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Goroutines</a></span></li><li><span><a href="#Example:-Concurrent-Clock-Server" data-toc-modified-id="Example:-Concurrent-Clock-Server-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Example: Concurrent Clock Server</a></span></li><li><span><a href="#Example:-Concurrent-Echo-Server" data-toc-modified-id="Example:-Concurrent-Echo-Server-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Example: Concurrent Echo Server</a></span></li><li><span><a href="#Channels" data-toc-modified-id="Channels-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Channels</a></span><ul class="toc-item"><li><span><a href="#Unbuffered-Channels" data-toc-modified-id="Unbuffered-Channels-8.4.1"><span class="toc-item-num">8.4.1&nbsp;&nbsp;</span>Unbuffered Channels</a></span></li><li><span><a href="#Pipelines" data-toc-modified-id="Pipelines-8.4.2"><span class="toc-item-num">8.4.2&nbsp;&nbsp;</span>Pipelines</a></span></li><li><span><a href="#Unidirectional-Channel-Type" data-toc-modified-id="Unidirectional-Channel-Type-8.4.3"><span class="toc-item-num">8.4.3&nbsp;&nbsp;</span>Unidirectional Channel Type</a></span></li><li><span><a href="#Buffered-Channels" data-toc-modified-id="Buffered-Channels-8.4.4"><span class="toc-item-num">8.4.4&nbsp;&nbsp;</span>Buffered Channels</a></span></li></ul></li><li><span><a href="#Looping-in-Parallel" data-toc-modified-id="Looping-in-Parallel-8.5"><span class="toc-item-num">8.5&nbsp;&nbsp;</span>Looping in Parallel</a></span></li><li><span><a href="#Example:-Concurrent-Web-Crawler" data-toc-modified-id="Example:-Concurrent-Web-Crawler-8.6"><span class="toc-item-num">8.6&nbsp;&nbsp;</span>Example: Concurrent Web Crawler</a></span></li><li><span><a href="#Multiplexing-with-select" data-toc-modified-id="Multiplexing-with-select-8.7"><span class="toc-item-num">8.7&nbsp;&nbsp;</span>Multiplexing with select</a></span></li><li><span><a href="#Example:-Concurrent-Directory-Travsal" data-toc-modified-id="Example:-Concurrent-Directory-Travsal-8.8"><span class="toc-item-num">8.8&nbsp;&nbsp;</span>Example: Concurrent Directory Travsal</a></span></li><li><span><a href="#Cancellation" data-toc-modified-id="Cancellation-8.9"><span class="toc-item-num">8.9&nbsp;&nbsp;</span>Cancellation</a></span></li><li><span><a href="#Example:-Chat-Server" data-toc-modified-id="Example:-Chat-Server-8.10"><span class="toc-item-num">8.10&nbsp;&nbsp;</span>Example: Chat Server</a></span></li></ul></li></ul></div>

# Goroutings and Channels

## Goroutines

In [22]:
//goroutine
import ("fmt";"time")
func f(i int) {
    go fmt.Println("Hello, World:",i)
    //fmt.Println("Hello, World:",i)
}
%%
for i:=0;i<10;i++ {
    go f(i)
    f(i)
}
time.Sleep(100* time.Millisecond)
//time.Sleep(1* time.Millisecond)

Hello, World: 0
Hello, World: 9
Hello, World: 7
Hello, World: 8
Hello, World: 8
Hello, World: 5
Hello, World: 6
Hello, World: 9
Hello, World: 5
Hello, World: 4
Hello, World: 1
Hello, World: 0
Hello, World: 7
Hello, World: 3
Hello, World: 6
Hello, World: 3
Hello, World: 4
Hello, World: 2
Hello, World: 1
Hello, World: 2


In [24]:
//gopl.io/ch8/spinner
import "time"
func main(){
    go spinner(100 * time.Millisecond)
    //spinner(100 * time.Millisecond)
    const n = 47
    //const n = 47
    fibN := fib(n) // slow
    fmt.Printf("\rFibonacci(%d) = %d\n", n, fibN)
}

func spinner(delay time.Duration) {
    for {
        for _, r := range `-\|/` {
            fmt.Printf("\r%c",r)
            time.Sleep(delay)
        }
    }
}

func fib(x int) int {
    if (x < 2) {
        return x
    }
    return fib(x-1) + fib(x-2)
}
//main()

Fibonacci(47) = 2971215073


## Example: Concurrent Clock Server
* gopl.io/ch8/clock1 : sequential clock server writing the current time to the client one per second
  * handleConn(conn)
* gopl.io/ch8/clock2 : handle connections concurrently
  * go handleConn(conn)

In [58]:
// Clock1 is a TCP server that periodically writes the time.
//package main
import (
	"io"
	"log"
	"net"
	"time"
)

func main() {
	listener, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	for {
		conn, err := listener.Accept()
		if err != nil {
			log.Print(err) // e.g., connection aborted
			continue
		}
		handleConn(conn) // handle one connection at a time
	}
}

func handleConn(c net.Conn) {
	defer c.Close()
	for {
        //f := "20060102150405\n"
        f := "15:04:05\n"
        _, err := io.WriteString(c, time.Now().Format(f)) 
		if err != nil {
			return // e.g., client disconnected
		}
		time.Sleep(1 * time.Second)
	}
}
//main()

^C
signal: interrupt


In [61]:
!nc localhost 8000

09:25:10
09:25:11
09:25:12
09:25:13
09:25:14
09:25:15
09:25:16
09:25:17


^C
signal: interrupt


In [62]:
// Netcat1 is a read-only TCP client.
//package main

import (
	"io"
	"log"
	"net"
	"os"
)

func main() {
	conn, err := net.Dial("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	defer conn.Close()
	mustCopy(os.Stdout, conn)
}

func mustCopy(dst io.Writer, src io.Reader) {
	if _, err := io.Copy(dst, src); err != nil {
		log.Fatal(err)
	}
}
//main()
//!-

09:25:32
09:25:33
09:25:34
09:25:35
09:25:36
09:25:37


^C
signal: interrupt


In [64]:
//gopl.io/ch8/clock2
import ("net";"log"; "io"; "time")
func main() {
  listener, err := net.Listen("tcp", "localhost:8000")
  if  err != nil {
    log.Fatal(err)
  }
  for {
    conn, err := listener.Accept()
    if err != nil {
        log.Print(err) // e.g., connection aborted
    }
    go handleConn(conn) // handle connections concurrently
  }
}

func handleConn(c net.Conn) {
	defer c.Close()
	for {
		_, err := io.WriteString(c, time.Now().Format("15:04:05\n"))
		if err != nil {
			return // e.g., client disconnected
		}
		time.Sleep(1 * time.Second)
	}
}
//main()

^C
signal: interrupt


## Example: Concurrent Echo Server
* reverb1
  * go handleConn
  * echo()
* reverb2
  * go handleConn
  * go echo()

In [65]:
// Reverb1 is a TCP server that simulates an echo.
//package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
	"strings"
	"time"
)

//!+
func echo(c net.Conn, shout string, delay time.Duration) {
	fmt.Fprintln(c, "\t", strings.ToUpper(shout))
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", shout)
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", strings.ToLower(shout))
}

func handleConn(c net.Conn) {
	input := bufio.NewScanner(c)
	for input.Scan() {
		echo(c, input.Text(), 1*time.Second)
	}
	// NOTE: ignoring potential errors from input.Err()
	c.Close()
}



In [66]:
//go handleConn(conn)
func main() {
	l, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	for {
		conn, err := l.Accept()
		if err != nil {
			log.Print(err) // e.g., connection aborted
			continue
		}
		go handleConn(conn)
	}
}


^C
signal: interrupt


In [ ]:
//main()

In [68]:
//gopl.io/ch8/netcat2
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 223.

// Netcat is a simple read/write client for TCP servers.
//package main

import (
	"io"
	"log"
	"net"
	"os"
)

//!+
func main() {
	conn, err := net.Dial("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	defer conn.Close()
	go mustCopy(os.Stdout, conn)
	mustCopy(conn, os.Stdin)
}

//!-

func mustCopy(dst io.Writer, src io.Reader) {
	if _, err := io.Copy(dst, src); err != nil {
		log.Fatal(err)
	}
}
//main()

12:12:45
12:12:46
12:12:47
12:12:48
12:12:49
12:12:50
12:12:51
12:12:52
12:12:53
12:12:54
12:12:55
12:12:56


^C
signal: interrupt


In [91]:
// Reverb2 is a TCP server that simulates an echo.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 224.

// Reverb2 is a TCP server that simulates an echo.
//package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
	"strings"
	"time"
)

func echo(c net.Conn, shout string, delay time.Duration) {
	fmt.Fprintln(c, "\t", strings.ToUpper(shout))
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", shout)
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", strings.ToLower(shout))
}

//!+
func handleConn(c net.Conn) {
	input := bufio.NewScanner(c)
	for input.Scan() {
		go echo(c, input.Text(), 1*time.Second)
	}
	// NOTE: ignoring potential errors from input.Err()
	c.Close()
}



In [92]:
// go handleConn()
import ("net"
        "log"
       )
func main() {
	l, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	for {
		conn, err := l.Accept()
		if err != nil {
			log.Print(err) // e.g., connection aborted
			continue
		}
		go handleConn(conn)
	}
}


^C
signal: interrupt


In [ ]:
//main()

## Channels

In [3]:
import "fmt"
%%
ch := make(chan int) // ch has type 'chan int'
go func() {ch <- 9}()
//ch <- 9
x := <- ch
fmt.Println(x)
close(ch) 

9


### Unbuffered Channels

In [5]:
//gopl.io/ch8/netcat3
// Netcat is a simple read/write client for TCP servers.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 227.


//package main

import (
	"io"
	"log"
	"net"
	"os"
)

//!+
func main() {
	conn, err := net.Dial("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	done := make(chan struct{})
	go func() {
		io.Copy(os.Stdout, conn) // NOTE: ignoring errors
		log.Println("done")
		done <- struct{}{} // signal the main goroutine
	}()
	mustCopy(conn, os.Stdin)
	conn.Close()
	<-done // wait for background goroutine to finish
}

//!-

func mustCopy(dst io.Writer, src io.Reader) {
	if _, err := io.Copy(dst, src); err != nil {
		log.Fatal(err)
	}
}
//main()

2024/11/26 12:31:20 dial tcp [::1]:8000: connect: connection refused
exit status 1


### Pipelines

In [7]:
// Pipeline1 demonstrates an infinite 3-stage pipeline.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 228.


//package main

import "fmt"

//!+
func main() {
	naturals := make(chan int)
	squares := make(chan int)

	// Counter
	go func() {
		for x := 0; x<100; x++ {
			naturals <- x
		}
        //close(naturals)
	}()

	// Squarer
	go func() {
		for {
			x := <-naturals
			squares <- x * x
		}
	}()

	// Printer (in main goroutine)
	for {
		fmt.Println(<-squares)
	}
}

//main()
//!-

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


fatal error: all goroutines are asleep - deadlock!

goroutine 1 [chan receive]:
main.main()
	 [[ Cell [7] Line 35 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_1a552118/main.go:38 +0xe0

goroutine 5 [chan receive]:
main.main.func2()
	 [[ Cell [7] Line 28 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_1a552118/main.go:31 +0x34
created by main.main in goroutine 1
	 [[ Cell [7] Line 26 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_1a552118/main.go:29 +0xd0
exit status 2


In [8]:
// Pipeline2 demonstrates a finite 3-stage pipeline.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 229.


//package main

import "fmt"

//!+
func main() {
	naturals := make(chan int)
	squares := make(chan int)

	// Counter
	go func() {
		for x := 0; x < 100; x++ {
			naturals <- x
		}
		close(naturals)
	}()

	// Squarer
	go func() {
		for x := range naturals {
			squares <- x * x
		}
		close(squares)
	}()

	// Printer (in main goroutine)
	for x := range squares {
		fmt.Println(x)
	}
}

//main()
//!-

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


### Unidirectional Channel Type

In [9]:
// Pipeline3 demonstrates a finite 3-stage pipeline
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 231.


// with range, close, and unidirectional channel types.
//package main

import "fmt"

//!+
func counter(out chan<- int) {
	for x := 0; x < 100; x++ {
		out <- x
	}
	close(out)
}

func squarer(out chan<- int, in <-chan int) {
	for v := range in {
		out <- v * v
	}
	close(out)
}

func printer(in <-chan int) {
	for v := range in {
		fmt.Println(v)
	}
}

func main() {
	naturals := make(chan int)
	squares := make(chan int)

	go counter(naturals)
	go squarer(squares, naturals)
	printer(squares)
}

//!-

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


In [3]:
//main()

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


### Buffered Channels

In [10]:
var ch chan string
func chinit() {
    ch = make(chan string, 3)
    ch <- "A"
    ch <- "B"
    ch <- "C"
}


In [11]:
%%
chinit()
fmt.Println(<- ch) // "A"
fmt.Println(cap(ch)) // "3"
fmt.Println(len(ch)) // "2"
fmt.Println(<- ch) // "B"
fmt.Println(<- ch) // "C"

A
3
2
B
C


In [12]:
func mirroredQuery() string {
    responses := make(chan string,3)
    go func() { responses <- request("asia.gopl.io")}()
    go func() { responses <- request("europe.gopl.io")}()
    go func() { responses <- request("americas.gopl.io")}()
    return <- responses // return the quickest response
}

# gonb_1a552118 

 

 Cell[12]: Line 3 
 ./main.go:30:30: undefined: request
 

func mirroredQuery() string {
 responses := make(chan string,3)
 go func() { responses <- request("asia.gopl.io")}()
 go func() { responses <- request("europe.gopl.io")}()
 go func() { responses <- request("americas.gopl.io")}()

 

 

 Cell[12]: Line 4 
 ./main.go:31:30: undefined: request
 
func mirroredQuery() string {
 responses := make(chan string,3)
 go func() { responses <- request("asia.gopl.io")}()
 go func() { responses <- request("europe.gopl.io")}()
 go func() { responses <- request("americas.gopl.io")}()
 return <- responses // return the quickest response

 

 

 Cell[12]: Line 5 
 ./main.go:32:30: undefined: request
 
 responses := make(chan string,3)
 go func() { responses <- request("asia.gopl.io")}()
 go func() { responses <- request("europe.gopl.io")}()
 go func() { responses <- request("americas.gopl.io")}()
 return <- responses // return the quickest response
}

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_1a552118/gonb_1a552118": exit status 1

## Looping in Parallel

In [13]:
//gopl.io/ch8/thumbnail
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 234.

// The thumbnail package produces thumbnail-size images from
// larger images.  Only JPEG images are currently supported.
//package thumbnail

import (
	"fmt"
	"image"
	"image/jpeg"
	"io"
	"os"
	"path/filepath"
	"strings"
)

// Image returns a thumbnail-size version of src.
func Image(src image.Image) image.Image {
	// Compute thumbnail size, preserving aspect ratio.
	xs := src.Bounds().Size().X
	ys := src.Bounds().Size().Y
	width, height := 128, 128
	if aspect := float64(xs) / float64(ys); aspect < 1.0 {
		width = int(128 * aspect) // portrait
	} else {
		height = int(128 / aspect) // landscape
	}
	xscale := float64(xs) / float64(width)
	yscale := float64(ys) / float64(height)

	dst := image.NewRGBA(image.Rect(0, 0, width, height))

	// a very crude scaling algorithm
	for x := 0; x < width; x++ {
		for y := 0; y < height; y++ {
			srcx := int(float64(x) * xscale)
			srcy := int(float64(y) * yscale)
			dst.Set(x, y, src.At(srcx, srcy))
		}
	}
	return dst
}

// ImageStream reads an image from r and
// writes a thumbnail-size version of it to w.
func ImageStream(w io.Writer, r io.Reader) error {
	src, _, err := image.Decode(r)
	if err != nil {
		return err
	}
	dst := Image(src)
	return jpeg.Encode(w, dst, nil)
}

// ImageFile2 reads an image from infile and writes
// a thumbnail-size version of it to outfile.
func ImageFile2(outfile, infile string) (err error) {
	in, err := os.Open(infile)
	if err != nil {
		return err
	}
	defer in.Close()

	out, err := os.Create(outfile)
	if err != nil {
		return err
	}

	if err := ImageStream(out, in); err != nil {
		out.Close()
		return fmt.Errorf("scaling %s to %s: %s", infile, outfile, err)
	}
	return out.Close()
}

// ImageFile reads an image from infile and writes
// a thumbnail-size version of it in the same directory.
// It returns the generated file name, e.g. "foo.thumb.jpeg".
func ImageFile(infile string) (string, error) {
	ext := filepath.Ext(infile) // e.g., ".jpg", ".JPEG"
	outfile := strings.TrimSuffix(infile, ext) + ".thumb" + ext
	return outfile, ImageFile2(outfile, infile)
}

In [14]:
// gopl.io/ch8/thumbnail_test
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// This file is just a place to put example code from the book.
// It does not actually run any code in gopl.io/ch8/thumbnail.

//package thumbnail_test

import (
	"log"
	"os"
	"sync"

	"gopl.io/ch8/thumbnail"
)

//!+1
// makeThumbnails makes thumbnails of the specified files.
func makeThumbnails(filenames []string) {
	for _, f := range filenames {
		if _, err := thumbnail.ImageFile(f); err != nil {
			log.Println(err)
		}
	}
}

 

In [15]:
// makeThumbnails2
// NOTE: incorrect!
func makeThumbnails2(filenames []string) {
	for _, f := range filenames {
		go thumbnail.ImageFile(f) // NOTE: ignoring errors
	}
}

In [16]:
// makeThumbnails3 makes thumbnails of the specified files in parallel.
func makeThumbnails3(filenames []string) {
	ch := make(chan struct{})
	for _, f := range filenames {
		go func(f string) {
			thumbnail.ImageFile(f) // NOTE: ignoring errors
			ch <- struct{}{}
		}(f)
	}

	// Wait for goroutines to complete.
	for range filenames {
		<-ch
	}
}


In [17]:
// makeThumbnails4 makes thumbnails for the specified files in parallel.
// It returns an error if any step failed.
func makeThumbnails4(filenames []string) error {
	errors := make(chan error)

	for _, f := range filenames {
		go func(f string) {
			_, err := thumbnail.ImageFile(f)
			errors <- err
		}(f)
	}

	for range filenames {
		if err := <-errors; err != nil {
			return err // NOTE: incorrect: goroutine leak!
		}
	}

	return nil
}

In [18]:
// makeThumbnails5 makes thumbnails for the specified files in parallel.
// It returns the generated file names in an arbitrary order,
// or an error if any step failed.
func makeThumbnails5(filenames []string) (thumbfiles []string, err error) {
	type item struct {
		thumbfile string
		err       error
	}

	ch := make(chan item, len(filenames))
	for _, f := range filenames {
		go func(f string) {
			var it item
			it.thumbfile, it.err = thumbnail.ImageFile(f)
			ch <- it
		}(f)
	}

	for range filenames {
		it := <-ch
		if it.err != nil {
			return nil, it.err
		}
		thumbfiles = append(thumbfiles, it.thumbfile)
	}

	return thumbfiles, nil
}


In [19]:
// makeThumbnails6 makes thumbnails for each file received from the channel.
// It returns the number of bytes occupied by the files it creates.
func makeThumbnails6(filenames <-chan string) int64 {
	sizes := make(chan int64)
	var wg sync.WaitGroup // number of working goroutines
	for f := range filenames {
		wg.Add(1)
		// worker
		go func(f string) {
			defer wg.Done()
			thumb, err := thumbnail.ImageFile(f)
			if err != nil {
				log.Println(err)
				return
			}
			info, _ := os.Stat(thumb) // OK to ignore error
			sizes <- info.Size()
		}(f)
	}

	// closer
	go func() {
		wg.Wait()
		close(sizes)
	}()

	var total int64
	for size := range sizes {
		total += size
	}
	return total
}

## Example: Concurrent Web Crawler

* A simple web crawler exploring the link graph in BFS
* concurrent and independent calls to crawl 
* exploit the I/O parallelism
  * crawl1 : unbounded parallelism
  * crawl2 : limited parallelism with counting semaphore
  * crawl3 : at most 20 HTTP reequests active concurrently

In [25]:
// Crawl1 crawls web links starting with the command-line arguments.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 240.


//
// This version quickly exhausts available file descriptors
// due to excessive concurrent calls to links.Extract.
//
// Also, it never terminates because the worklist is never closed.
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch5/links"
)

//!+crawl
func crawl(url string) []string {
	fmt.Println(url)
	list, err := links.Extract(url)
	if err != nil {
		log.Print(err)
	}
	return list
}

//!-crawl

//!+main
%args http://gopl.io/
func main() {
	worklist := make(chan []string)

	// Start with the command-line arguments.
	go func() { worklist <- os.Args[1:] }()

	// Crawl the web concurrently.
	seen := make(map[string]bool)
	for list := range worklist {
		for _, link := range list {
			if !seen[link] {
				seen[link] = true
				go func(link string) {
					worklist <- crawl(link)
				}(link)
			}
		}
	}
}

//!-main

/*
//!+output
$ go build gopl.io/ch8/crawl1
$ ./crawl1 http://gopl.io/
http://gopl.io/
https://golang.org/help/

https://golang.org/doc/
https://golang.org/blog/
...
2015/07/15 18:22:12 Get ...: dial tcp: lookup blog.golang.org: no such host
2015/07/15 18:22:12 Get ...: dial tcp 23.21.222.120:443: socket:
                                                        too many open files
...
//!-output
*/

http://gopl.io/
http://www.barnesandnoble.com/w/1121601944
http://www.amazon.com/dp/020161586X?tracking_id=disfordig-20
http://www.informit.com/store/go-programming-language-9780134190440
http://www.amazon.com/dp/0134190440
http://www.gopl.io/errata.html
http://www.gopl.io/ch1.pdf
http://golang.org/s/oracle-user-manual
http://www.gopl.io/reviews.html
https://github.com/adonovan/gopl.io/
https://github.com/golang/tools/blob/master/refactor/eg/eg.go
http://golang.org/lib/godoc/analysis/help.html
https://github.com/golang/tools/blob/master/refactor/rename/rename.go
http://www.amazon.com/dp/0131103628?tracking_id=disfordig-20
http://www.gopl.io/translations.html
https://github.com/golang/proposal/blob/master/design/12416-cgo-pointers.md
http://www.amazon.com/Programming-Language-Addison-Wesley-Professional-Computing/product-reviews/0134190440/ref=cm_cr_dp_see_all_summary
http://www.pearsonapac.com/
https://www.usenix.org/system/files/login/articles/login_dec15_17_books.pdf
http://www.infoq

2024/11/26 13:49:17 getting http://www.barnesandnoble.com/w/1121601944: 403 Forbidden


https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496
https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088


2024/11/26 13:49:17 Get "http://www.pearsonapac.com/": dial tcp: lookup www.pearsonapac.com: no such host
2024/11/26 13:49:17 getting https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088: 403 Forbidden


http://www.maruzen.co.jp/corp/en/services/publishing.html#container
http://yushodo.maruzen.co.jp
https://www.tenlong.com.tw/announcements/313
https://www.tenlong.com.tw/
https://www.tenlong.com.tw/announcements/314
https://www.tenlong.com.tw/terms
https://www.tenlong.com.tw/categories/children
https://www.tenlong.com.tw/announcements/310
https://www.tenlong.com.tw/announcements/307
https://www.tenlong.com.tw/announcements/305
https://www.tenlong.com.tw/announcements/302
https://www.tenlong.com.tw/events/2023-bts-booklist
https://www.tenlong.com.tw/events/culture-point
https://www.tenlong.com.tw/events/enservice
https://www.tenlong.com.tw/events/5g-6g
https://www.tenlong.com.tw/events/106
https://www.tenlong.com.tw/events/subcribe
https://www.tenlong.com.tw/events/shoplinetw
https://www.tenlong.com.tw/publishers/14
https://www.tenlong.com.tw/publishers/15
https://www.tenlong.com.tw/categories/programming-languages
https://www.tenlong.com.tw/publishers/60
https://www.tenlong.com.tw/categ

...

2024/11/26 13:49:29 Get "javascript:showlink('http://www.williamspublishing.com/cgi-bin/more.cgi?isbn=5-8459-0863-9')": unsupported protocol scheme "javascript"


javascript:showlink('http://www.williamspublishing.com/cgi-bin/more.cgi?isbn=5-8459-0863-9')
http://www.williamspublishing.com/PDF/978-5-8459-1793-5/part.pdf


2024/11/26 13:49:30 Get "javascript:showlink('http://www.williamspublishing.com/cgi-bin/more.cgi?isbn=978-5-8459-1793-5')": unsupported protocol scheme "javascript"
2024/11/26 13:49:30 getting http://www.williamspublishing.com/cgi-bin/buy.cgi?isbn=978-5-8459-1838-3&next=yes: 520 


http://www.williamspublishing.com/Books/5-8459-0863-9.html#ogl
http://www.williamspublishing.com/Books/thumb/big/978-5-8459-1793-5.jpg
http://www.williamspublishing.com/cgi-bin/buy.cgi?isbn=978-5-8459-1793-5
http://www.williamspublishing.com/Books/978-5-8459-1793-5.html#ogl
javascript:showlink('http://www.williamspublishing.com/cgi-bin/more.cgi?isbn=978-5-8459-1793-5')
https://shtonda.blogspot.com/2012/02/core-python-applications-programming.html
http://www.williamspublishing.com/cgi-bin/series.cgi?seria=���������� �������������#list
http://www.williamspublishing.com/Books/978-5-8459-1801-7.html
http://www.williamspublishing.com/PDF/978-5-8459-1793-5/autor.pdf
http://www.williamspublishing.com/PDF/978-5-8459-1793-5/preface.pdf
https://network.creativecommons.org/call-for-stories-open-glam-in-underrepresented-communities/
https://github.com/creativecommons/global-network-strategy/blob/main/docs/Global_Network_Membership_Charter.md#codes-of-conduct
https://thegreats.co/artworks/the-futur

2024/11/26 13:49:30 getting https://www.linkedin.com/jobs/director-jobs?trk=organization_guest-browse_jobs: 429 Too Many Requests
2024/11/26 13:49:30 getting https://www.linkedin.com/jobs/associate-jobs?trk=organization_guest_linkster_link: 429 Too Many Requests
^C
signal: interrupt


* Bracketed links.Extract by operations to acquire and release a token
* Ensuring at most 20 calls to it are active at one time

In [26]:
// Crawl2 crawls web links starting with the command-line arguments.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 241.


//
// This version uses a buffered channel as a counting semaphore
// to limit the number of concurrent calls to links.Extract.
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch5/links"
)

//!+sema
// tokens is a counting semaphore used to
// enforce a limit of 20 concurrent requests.
var tokens = make(chan struct{}, 20)

func crawl(url string) []string {
	fmt.Println(url)
	tokens <- struct{}{} // acquire a token
	list, err := links.Extract(url)
	<-tokens // release the token

	if err != nil {
		log.Print(err)
	}
	return list
}

//!-sema

//!+
%args http://gopl.io/
func main() {
	worklist := make(chan []string)
	var n int // number of pending sends to worklist

	// Start with the command-line arguments.
	n++
	go func() { worklist <- os.Args[1:] }()

	// Crawl the web concurrently.
	seen := make(map[string]bool)
	for ; n > 0; n-- {
		list := <-worklist
		for _, link := range list {
			if !seen[link] {
				seen[link] = true
				n++
				go func(link string) {
					worklist <- crawl(link)
				}(link)
			}
		}
	}
}

//!-

http://gopl.io/
http://www.amazon.com/dp/020161586X?tracking_id=disfordig-20
http://www.gopl.io/translations.html
http://www.informit.com/store/go-programming-language-9780134190440
http://www.gopl.io/errata.html
https://github.com/adonovan/gopl.io/
http://www.gopl.io/ch1.pdf
http://www.amazon.com/dp/0134190440
http://www.barnesandnoble.com/w/1121601944
http://golang.org/s/oracle-user-manual
https://github.com/golang/tools/blob/master/refactor/eg/eg.go
http://golang.org/lib/godoc/analysis/help.html
http://www.gopl.io/reviews.html
https://github.com/golang/tools/blob/master/refactor/rename/rename.go
http://www.amazon.com/dp/0131103628?tracking_id=disfordig-20
http://www.pearsonapac.com/
http://www.maruzen.co.jp/corp/en/services/publishing.html
http://helion.pl/ksiazki/jezyk-go-poznaj-i-programuj-alan-a-a-donovan-brian-w-kernighan,jgopop.htm
http://helion.pl/
http://www.amazon.co.jp/exec/obidos/ASIN/4621300253
https://novatec.com.br/livros/linguagem-de-programacao-go/
https://www.tenlong

2024/11/26 13:50:52 Get "http://www.pearsonapac.com/": dial tcp: lookup www.pearsonapac.com: no such host


https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496
https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088
https://www.amazon.co.jp/gp/help/customer/display.html/ref=footer_privacy/376-1267051-7966065?ie=UTF8&nodeId=643000
https://www.amazon.co.jp/gp/help/customer/display.html/ref=footer_cou/376-1267051-7966065?ie=UTF8&nodeId=643006
http://www.maruzen.co.jp/corp/en/services/publishing.html#container
http://yushodo.maruzen.co.jp


2024/11/26 13:50:52 getting http://www.barnesandnoble.com/w/1121601944: 403 Forbidden


https://www.tenlong.com.tw/terms
https://www.tenlong.com.tw/
https://www.tenlong.com.tw/announcements/314
https://www.tenlong.com.tw/announcements/313
https://www.tenlong.com.tw/announcements/310
https://www.tenlong.com.tw/announcements/307
https://www.tenlong.com.tw/announcements/305
https://www.tenlong.com.tw/announcements/302
https://www.tenlong.com.tw/events/2023-bts-booklist
https://www.tenlong.com.tw/events/culture-point
https://www.tenlong.com.tw/events/enservice
https://www.tenlong.com.tw/events/5g-6g
https://www.tenlong.com.tw/events/106
https://www.tenlong.com.tw/events/subcribe
https://www.tenlong.com.tw/events/shoplinetw
https://www.tenlong.com.tw/events/shoeline-backend
https://www.tenlong.com.tw/categories/agile
https://www.tenlong.com.tw/categories/devops
https://www.tenlong.com.tw/categories/levelup
https://www.tenlong.com.tw/categories/python
https://www.tenlong.com.tw/categories/langchain
https://www.tenlong.com.tw/categories/deep-learning
https://www.tenlong.com.tw/c

...

https://saedu.naver.com/adbiz/searchad/intro.nhn
http://book.interpark.com/order/cartlist.do?_method=cartList
http://commevent.interpark.com/event/promotion/interparkLotteCard?view=pc
http://book.interpark.com/bookPark/sitemap/CateIncludeAll.jsp?fn=2009/service/sub03_1_bookpinion.html
http://book.interpark.com/service/PerfectService.do?_method=detail&sc.ntcNo=44&bid1=17_main&bid2=bottom&bid3=benefit&bid4=06
http://incorp.interpark.com/member/MemberGoodService.do?_method=GoodServiceChk
http://book.interpark.com/event/eventCoupon.do?_method=detail
http://book.interpark.com/bookPark/html/book.html?bnid1=book_2015&bnid2=locat&bnid3=cate&bnid4=home
http://book.interpark.com/bookPark/html/book_kor.html?bnid1=book_2015&bnid2=locat&bnid3=cate&bnid4=01
http://book.interpark.com/bookPark/html/foreign_main.html?bnid1=book_2015&bnid2=locat&bnid3=cate&bnid4=01
http://book.interpark.com/gen_display/book/gen_category_028005.html?bnid1=book_2015&bnid2=locat&bnid3=cate&bnid4=02
http://book.interpark.co

2024/11/26 13:51:03 getting http://www.acornpub.co.kr/download/go-programming: 403 Forbidden


https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=240284531
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=5919&BrowseTarget=List
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=5915
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1108
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1383
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=5914&BrowseTarget=List
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=9280&BrowseTarget=List
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=74
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1196
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=55889
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=517
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1322
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1230
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=13789
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=656
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=336
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=2913
https:/

https://www.aladin.co.kr/shop/wbrowse.aspx?CID=4395
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=34582
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=987
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=2551
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=8257
https://www.aladin.co.kr/shop/wbrowse.aspx?CID=798
https://www.gotop.com.tw/certification/Privacy.aspx
https://www.gotop.com.tw/certification/Items.aspx?type=microsoft&id=11
https://www.gotop.com.tw/certification/contact01.aspx
https://www.gotop.com.tw/certification/About01.aspx
https://www.gotop.com.tw/certification/itemb.aspx
https://www.gotop.com.tw/certification/MS_its.html
https://www.gotop.com.tw/certification/ItemH.aspx
https://www.gotop.com.tw/certification/ItemM.aspx
https://www.gotop.com.tw/certification/Items.aspx?type=autodesk&id=1
https://www.gotop.com.tw/certification/Items.aspx?type=microsoft&id=2
https://www.gotop.com.tw/certification/Items.aspx?type=adobe&id=13
https://www.gotop.com.tw/certification/Items.aspx?typ

^C
signal: interrupt


In [ ]:
// Crawl3 crawls web links starting with the command-line arguments.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 243.

//
// This version uses bounded parallelism.
// For simplicity, it does not address the termination problem.
//
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch5/links"
)

func crawl(url string) []string {
	fmt.Println(url)
	list, err := links.Extract(url)
	if err != nil {
		log.Print(err)
	}
	return list
}

//!+
%args http://gopl.io/
func main() {
	worklist := make(chan []string)  // lists of URLs, may have duplicates
	unseenLinks := make(chan string) // de-duplicated URLs

	// Add command-line arguments to worklist.
	go func() { worklist <- os.Args[1:] }()

	// Create 20 crawler goroutines to fetch each unseen link.
	for i := 0; i < 20; i++ {
		go func() {
			for link := range unseenLinks {
				foundLinks := crawl(link)
				go func() { worklist <- foundLinks }()
			}
		}()
	}

	// The main goroutine de-duplicates worklist items
	// and sends the unseen ones to the crawlers.
	seen := make(map[string]bool)
	for list := range worklist {
		for _, link := range list {
			if !seen[link] {
				seen[link] = true
				unseenLinks <- link
			}
		}
	}
}

//!-

## Multiplexing with select
* Countdown for a rocket launch
* countdown1 : time.Tick returns a channel on which it sends events periodically
* countdown2 : abort the launch sequence by pressing the return key during the countdown
* countdown3 : abort the launch sequence by pressing the return key and print the countdown

In [27]:
// Countdown1 implements the countdown for a rocket launch.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 244.


//package main

import (
	"fmt"
	"time"
)

//!+
func main() {
	fmt.Println("Commencing countdown.")
	tick := time.Tick(2 * time.Second)
	for countdown := 10; countdown > 0; countdown-- {
		fmt.Println(countdown)
		<-tick
	}
	launch()
}

//!-

func launch() {
	fmt.Println("Lift off!")
}

Commencing countdown.
10
9
8
7
6
5
4
3
2
1
Lift off!


In [32]:
//main()

Commencing countdown.
10
9
8
7
6
5
4
3
2
1
Lift off!


In [22]:
// Countdown2 implements the countdown for a rocket launch.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 244.

//package main

import (
	"fmt"
	"os"
	"time"
)

//!+

func main() {
	// ...create abort channel...

	//!-

	//!+abort
	abort := make(chan struct{})
	go func() {
		os.Stdin.Read(make([]byte, 1)) // read a single byte
		abort <- struct{}{}
	}()
	//!-abort

	//!+
	fmt.Println("Commencing countdown.  Press return to abort.")
	select {
	case <-time.After(10 * time.Second):
		// Do nothing.
	case <-abort:
		fmt.Println("Launch aborted!")
		return
	}
	launch()
}

//!-

func launch() {
	fmt.Println("Lift off!")
}

Commencing countdown.  Press return to abort.
Lift off!


In [29]:
%%
ch := make(chan int,1)
for i:=0; i<10;i ++ {
    select {
    case  x := <- ch:
        fmt.Println(x) // "0" "2" "4" "6" "8"
    case  ch <- i:
    }
}

0
2
4
6
8


In [30]:
// Countdown3 implements the countdown for a rocket launch.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 246.


//package main

// NOTE: the ticker goroutine never terminates if the launch is aborted.
// This is a "goroutine leak".

import (
	"fmt"
	"os"
	"time"
)

//!+

func main() {
	// ...create abort channel...

	//!-

	abort := make(chan struct{})
	go func() {
		os.Stdin.Read(make([]byte, 1)) // read a single byte
		abort <- struct{}{}
	}()

	//!+
	fmt.Println("Commencing countdown.  Press return to abort.")
	tick := time.Tick(1 * time.Second)
	for countdown := 10; countdown > 0; countdown-- {
		fmt.Println(countdown)
		select {
		case <-tick:
			// Do nothing.
		case <-abort:
			fmt.Println("Launch aborted!")
			return
		}
	}
	launch()
}

//!-

func launch() {
	fmt.Println("Lift off!")
}

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_7d1e0af5/gonb_7d1e0af5": signal: interrupt

## Example: Concurrent Directory Travsal
* du1: disk usage without parallelism 
* du2: du1 with progress every 500ms
* du3: walkDir concurrently 

In [24]:
// The du1 command computes the disk usage of the files in a directory.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 247.

//!+main

//package main

import (
	"flag"
	"fmt"
	"io/ioutil"
	"os"
	"path/filepath"
)
%args /
func main() {
	// Determine the initial directories.
	flag.Parse()
	roots := flag.Args()
	if len(roots) == 0 {
		roots = []string{"."}
	}

	// Traverse the file tree.
	fileSizes := make(chan int64)
	go func() {
		for _, root := range roots {
			walkDir(root, fileSizes)
		}
		close(fileSizes)
	}()

	// Print the results.
	var nfiles, nbytes int64
	for size := range fileSizes {
		nfiles++
		nbytes += size
	}
	printDiskUsage(nfiles, nbytes)
}

func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

//!-main

//!+walkDir

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
func walkDir(dir string, fileSizes chan<- int64) {
	for _, entry := range dirents(dir) {
		if entry.IsDir() {
			subdir := filepath.Join(dir, entry.Name())
			walkDir(subdir, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
	}
}

// dirents returns the entries of directory dir.
func dirents(dir string) []os.FileInfo {
	entries, err := ioutil.ReadDir(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du1: %v\n", err)
		return nil
	}
	return entries
}

//!-walkDir

// The du1 variant uses two goroutines and
// prints the total after every file is found.

du1: open /Library/Application Support/Apple/AssetCache/Data: permission denied
du1: open /Library/Application Support/Apple/ParentalControls/Users: permission denied
du1: open /Library/Application Support/ApplePushService: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/126.0.6462.0: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/127.0.6490.0: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/128.0.6537.0: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/128.0.6597.0: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/130.0.6679.0: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/131.0.6776.0: permission denied
du1: open /Library/Application Support/Google/GoogleUpdater/132.0.6833.0/Crashpad: permission denied
du1: open /Library/Application Support/Microsoft/EdgeUpdater/128.0.2739.5/Crashpad: permission denied
du1

du1: open /System/Library/Templates/Data/private/var/spool/postfix/bounce: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/corrupt: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/defer: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/deferred: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/flush: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/hold: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/incoming: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/maildrop: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/private: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix/public: permission denied
du1: open /System/Library/Templates/Data/private/var/spool/postfix

du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/FamilyCircle: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.HomeKit: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.Safari: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.ap.adprivacyd: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.containermanagerd: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.findmy.fmfcore: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.findmy.fmipcore: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.findmy.imagecache: operation not permitted
du1: open /System/Volumes/Data/Users/skhuang/Library/Caches/com.apple.homed: operation not permitted
du1: open /System/Volumes/Data/Users/skh

In [38]:
os.Args = []string{"du1","/etc"}
main()

du1: open /etc/polkit-1/localauthority: permission denied
du1: open /etc/ssl/private: permission denied


1798 files  0.0 GB


In [31]:
// The du2 command computes the disk usage of the files in a directory.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 249.

//package main

// The du2 variant uses select and a time.Ticker
// to print the totals periodically if -v is set.

import (
	"flag"
	"fmt"
	"io/ioutil"
	"os"
	"path/filepath"
	"time"
)

//!+
var verbose = flag.Bool("v", false, "show verbose progress messages")
%args -v /
func main() {
	// ...start background goroutine...

	//!-
	// Determine the initial directories.
	flag.Parse()
	roots := flag.Args()
	if len(roots) == 0 {
		roots = []string{"."}
	}

	// Traverse the file tree.
	fileSizes := make(chan int64)
	go func() {
		for _, root := range roots {
			walkDir(root, fileSizes)
		}
		close(fileSizes)
	}()

	//!+
	// Print the results periodically.
	var tick <-chan time.Time
	if *verbose {
		tick = time.Tick(500 * time.Millisecond)
	}
	var nfiles, nbytes int64
loop:
	for {
		select {
		case size, ok := <-fileSizes:
			if !ok {
				break loop // fileSizes was closed
			}
			nfiles++
			nbytes += size
		case <-tick:
			printDiskUsage(nfiles, nbytes)
		}
	}
	printDiskUsage(nfiles, nbytes) // final totals
}

//!-

func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
func walkDir(dir string, fileSizes chan<- int64) {
	for _, entry := range dirents(dir) {
		if entry.IsDir() {
			subdir := filepath.Join(dir, entry.Name())
			walkDir(subdir, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
	}
}

// dirents returns the entries of directory dir.
func dirents(dir string) []os.FileInfo {
	entries, err := ioutil.ReadDir(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		return nil
	}
	return entries
}

21295 files  1.7 GB
42149 files  2.7 GB
63601 files  7.2 GB
89630 files  10.9 GB
114367 files  14.3 GB
139657 files  15.7 GB
162096 files  17.3 GB
190733 files  18.1 GB
218775 files  20.0 GB
244926 files  21.5 GB
267522 files  25.0 GB
294349 files  26.0 GB
327056 files  26.9 GB
356951 files  28.7 GB
373288 files  31.7 GB
390943 files  33.3 GB
410843 files  35.7 GB
428495 files  36.6 GB
441924 files  37.0 GB
448146 files  37.2 GB
467379 files  37.4 GB
482747 files  38.3 GB
505023 files  39.1 GB
522090 files  45.9 GB
544907 files  49.2 GB
569304 files  50.4 GB
594739 files  54.2 GB


du: open /Library/Application Support/Apple/AssetCache/Data: permission denied
du: open /Library/Application Support/Apple/ParentalControls/Users: permission denied
du: open /Library/Application Support/ApplePushService: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/126.0.6462.0: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/127.0.6490.0: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/128.0.6537.0: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/128.0.6597.0: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/130.0.6679.0: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/131.0.6776.0: permission denied
du: open /Library/Application Support/Google/GoogleUpdater/132.0.6833.0/Crashpad: permission denied
du: open /Library/Application Support/Microsoft/EdgeUpdater/128.0.2739.5/Crashpad: permission denied
du: open /Libr

610435 files  57.7 GB
622836 files  58.0 GB
646930 files  58.3 GB
660290 files  58.5 GB
686460 files  58.8 GB
697048 files  59.0 GB
722815 files  59.4 GB
732433 files  59.6 GB
761226 files  60.0 GB
787290 files  62.2 GB
804763 files  64.6 GB


du: open /Library/InstallerSandboxes/.PKInstallSandboxManager/2ABB92BA-C746-49B5-951E-1697D355B4D1.sandbox: permission denied
du: open /Library/InstallerSandboxes/.PKInstallSandboxManager/E330DD83-26EF-4D8E-A509-77E83C3197FE.activeSandbox: permission denied
du: open /Library/Trial: operation not permitted


817954 files  66.7 GB
832448 files  67.7 GB
845790 files  71.7 GB
846822 files  73.3 GB
863889 files  75.8 GB
878394 files  77.0 GB
897194 files  78.3 GB
927974 files  78.4 GB
952609 files  78.4 GB
971839 files  78.4 GB
988998 files  78.6 GB


...

du: open /System/Volumes/Data/Library/Trial: operation not permitted


1984994 files  161.4 GB
1986754 files  163.6 GB
1991381 files  166.1 GB
2004754 files  166.1 GB
2018693 files  167.3 GB
2034086 files  168.7 GB


du: open /System/Volumes/Data/System/Library/Caches/com.apple.coresymbolicationd: permission denied
du: open /System/Volumes/Data/Users/skhuang/.Trash: operation not permitted


2045398 files  169.3 GB
2066659 files  169.4 GB
2072293 files  170.3 GB
2081978 files  279.1 GB
2093478 files  279.2 GB
2106263 files  279.2 GB
2115800 files  279.3 GB


^C
signal: interrupt


In [1]:
// The du3 command computes the disk usage of the files in a directory
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 250.

//package main

// The du3 variant traverses all directories in parallel.
// It uses a concurrency-limiting counting semaphore
// to avoid opening too many files at once.

import (
	"flag"
	"fmt"
	"io/ioutil"
	"os"
	"path/filepath"
	"sync"
	"time"
)

var vFlag = flag.Bool("v", false, "show verbose progress messages")
%args -v /etc
//!+
func main() {
	// ...determine roots...

	//!-
	flag.Parse()

	// Determine the initial directories.
	roots := flag.Args()
	if len(roots) == 0 {
		roots = []string{"."}
	}

	//!+
	// Traverse each root of the file tree in parallel.
	fileSizes := make(chan int64)
	var n sync.WaitGroup
	for _, root := range roots {
		n.Add(1)
		go walkDir(root, &n, fileSizes)
	}
	go func() {
		n.Wait()
		close(fileSizes)
	}()
	//!-

	// Print the results periodically.
	var tick <-chan time.Time
	if *vFlag {
		tick = time.Tick(500 * time.Millisecond)
	}
	var nfiles, nbytes int64
loop:
	for {
		select {
		case size, ok := <-fileSizes:
			if !ok {
				break loop // fileSizes was closed
			}
			nfiles++
			nbytes += size
		case <-tick:
			printDiskUsage(nfiles, nbytes)
		}
	}

	printDiskUsage(nfiles, nbytes) // final totals
	//!+
	// ...select loop...
}

//!-

func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
//!+walkDir
func walkDir(dir string, n *sync.WaitGroup, fileSizes chan<- int64) {
	defer n.Done()
	for _, entry := range dirents(dir) {
		if entry.IsDir() {
			n.Add(1)
			subdir := filepath.Join(dir, entry.Name())
			go walkDir(subdir, n, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
	}
}

//!-walkDir

//!+sema
// sema is a counting semaphore for limiting concurrency in dirents.
var sema = make(chan struct{}, 20)

// dirents returns the entries of directory dir.
func dirents(dir string) []os.FileInfo {
	sema <- struct{}{}        // acquire token
	defer func() { <-sema }() // release token
	// ...
	//!-sema

	entries, err := ioutil.ReadDir(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		return nil
	}
	return entries
}

du: open /etc/cups/certs: permission denied


243 files  0.0 GB


## Cancellation

In [3]:
// The du4 command computes the disk usage of the files in a directory.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 251.

//package main

// The du4 variant includes cancellation:
// it terminates quickly when the user hits return.

import (
	"fmt"
	"os"
	"path/filepath"
	"sync"
	"time"
)

//!+1
var done = make(chan struct{})

func cancelled() bool {
	select {
	case <-done:
		return true
	default:
		return false
	}
}

In [4]:
// du4, main
%args /etc
func main() {
	// Determine the initial directories.
	roots := os.Args[1:]
	if len(roots) == 0 {
		roots = []string{"."}
	}

	//!+2
	// Cancel traversal when input is detected.
	go func() {
		os.Stdin.Read(make([]byte, 1)) // read a single byte
		close(done)
	}()
	//!-2

	// Traverse each root of the file tree in parallel.
	fileSizes := make(chan int64)
	var n sync.WaitGroup
	for _, root := range roots {
		n.Add(1)
		go walkDir(root, &n, fileSizes)
	}
	go func() {
		n.Wait()
		close(fileSizes)
	}()

	// Print the results periodically.
	tick := time.Tick(500 * time.Millisecond)
	var nfiles, nbytes int64
loop:
	//!+3
	for {
		select {
		case <-done:
			// Drain fileSizes to allow existing goroutines to finish.
			for range fileSizes {
				// Do nothing.
			}
			return
		case size, ok := <-fileSizes:
			// ...
			//!-3
			if !ok {
				break loop // fileSizes was closed
			}
			nfiles++
			nbytes += size
		case <-tick:
			printDiskUsage(nfiles, nbytes)
		}
	}
	printDiskUsage(nfiles, nbytes) // final totals
}



du: open /etc/cups/certs: permission denied


243 files  0.0 GB


In [5]:
//printDiskUsage, walkDir, dirents
func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
//!+4
func walkDir(dir string, n *sync.WaitGroup, fileSizes chan<- int64) {
	defer n.Done()
	if cancelled() {
		return
	}
	for _, entry := range dirents(dir) {
		// ...
		//!-4
		if entry.IsDir() {
			n.Add(1)
			subdir := filepath.Join(dir, entry.Name())
			go walkDir(subdir, n, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
		//!+4
	}
}

//!-4

var sema = make(chan struct{}, 20) // concurrency-limiting counting semaphore

// dirents returns the entries of directory dir.
//!+5
func dirents(dir string) []os.FileInfo {
	select {
	case sema <- struct{}{}: // acquire token
	case <-done:
		return nil // cancelled
	}
	defer func() { <-sema }() // release token

	// ...read directory...
	//!-5

	f, err := os.Open(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		return nil
	}
	defer f.Close()

	entries, err := f.Readdir(0) // 0 => no limit; read all entries
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		// Don't return: Readdir may return partial results.
	}
	return entries
}

## Example: Chat Server

In [6]:
// broadcaster
// Chat is a server that lets clients chat with each other.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 254.
//!+

//package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
)

//!+broadcaster
type client chan<- string // an outgoing message channel

var (
	entering = make(chan client)
	leaving  = make(chan client)
	messages = make(chan string) // all incoming client messages
)

func broadcaster() {
	clients := make(map[client]bool) // all connected clients
	for {
		select {
		case msg := <-messages:
			// Broadcast incoming message to all
			// clients' outgoing message channels.
			for cli := range clients {
				cli <- msg
			}

		case cli := <-entering:
			clients[cli] = true

		case cli := <-leaving:
			delete(clients, cli)
			close(cli)
		}
	}
}



In [7]:
//!+handleConn
//clientWriter and main
func clientWriter(conn net.Conn, ch <-chan string) {
	for msg := range ch {
		fmt.Fprintln(conn, msg) // NOTE: ignoring network errors
	}
}
func handleConn(conn net.Conn) {
	ch := make(chan string) // outgoing client messages
	go clientWriter(conn, ch)

	who := conn.RemoteAddr().String()
	ch <- "You are " + who
	messages <- who + " has arrived"
	entering <- ch

	input := bufio.NewScanner(conn)
	for input.Scan() {
		messages <- who + ": " + input.Text()
	}
	// NOTE: ignoring potential errors from input.Err()

	leaving <- ch
	messages <- who + " has left"
	conn.Close()
}



In [13]:


//!-handleConn

//!+main
func main() {
	listener, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}

	go broadcaster()
	for {
		conn, err := listener.Accept()
		if err != nil {
			log.Print(err)
			continue
		}
		go handleConn(conn)
	}
}

//!-main

^C
signal: interrupt
